In [1]:
from narrative_llm_agent.agents.kbase_agent import KBaseAgent
from narrative_llm_agent.agents.analyst import AnalystAgent
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew, Process
import config

In [2]:
def test_kbase_agent():
    agent = KBaseAgent("foo", None)
    assert agent._token == "foo"
    print("like totally passed.")

In [3]:
llm = ChatOpenAI(temperature=0, model="gpt-4",streaming=True,openai_api_key=config.api_key)

In [4]:
researcher = AnalystAgent(token=config.api_key, llm=llm)

In [5]:
# Create tasks for your agents
task1 = Task(
  description="""I have sequenced a new microbe. I have a fastq file of reads from paired end illumina sequencing.  
  Using KBase apps, I would like to understand if my newly sequenced microbe is a cysteine auxotroph. Which KBase apps should I use and how do I use them?
""",
  agent=researcher.agent
)
task2 = Task(
  description="""I have sequenced a new microbe. I have a fastq file of reads from paired end illumina sequencing.  
  I want to perform trimming and quality control, which KBase apps should I use?
""",
  agent=researcher.agent
)

In [6]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher.agent],
  tasks=[task1],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

In [7]:
# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

[DEBUG]: Working Agent: Computational Biologist and Geneticist
[INFO]: Starting Task: I have sequenced a new microbe. I have a fastq file of reads from paired end illumina sequencing.  
  Using KBase apps, I would like to understand if my newly sequenced microbe is a cysteine auxotroph. Which KBase apps should I use and how do I use them?



> Entering new CrewAgentExecutor chain...
Use Tool: Kbase app catalog retrieval tool, "Which KBase apps can help me analyze a fastq file from paired end illumina sequencing to determine if my newly sequenced microbe is a cysteine auxotroph?"  

{'query': 'Which KBase apps can help me analyze a fastq file from paired end illumina sequencing to determine if my newly sequenced microbe is a cysteine auxotroph?', 'chat_history': '', 'result': "Based on the additional context, the following KBase apps could be useful in analyzing a fastq file from paired-end Illumina sequencing to determine if your newly sequenced microbe is a cysteine auxotroph:\n\n1. '